In [1]:
import numpy as np
import pandas as pd

In [ ]:
stocks = pd.read

In [ ]:
import plotly.express as px
import pandas as pd

fig = px.scatter(stocks, x="risk", y="return", text=stocks.index.values, log_x=True, size_max=100, color="risk")
fig.update_traces(textposition='top center')
fig.update_layout(title_text='Sample', title_x=0.5)
fig.show()